# Generate instance of Knapsack

In [3]:
#Suppose we have a instance of 5 items


import random
from pyscipopt import Model, quicksum

# -----------------------------
# 1. Generate Random Instance
# -----------------------------
def generate_knapsack_instance(num_items=5, max_weight=20, max_value=50, seed=40):
    random.seed(seed)
    weights = [random.randint(1, max_weight) for _ in range(num_items)]
    values = [random.randint(1, max_value) for _ in range(num_items)]
    capacity = int(sum(weights) * 0.6)  # set capacity to 60% of total weight
    return weights, values, capacity

# -----------------------------

weights, values, capacity = generate_knapsack_instance()

# Solve instance using IP

In [4]:
# 2. Build and Solve in SCIP
# -----------------------------
def solve_knapsack(weights, values, capacity):
    n = len(weights)
    model = Model("0-1_Knapsack")

    # Decision variables: x[i] = 1 if item i is chosen
    x = {i: model.addVar(vtype="B", name=f"x_{i}") for i in range(n)}

    # Objective: maximize value
    model.setObjective(quicksum(values[i] * x[i] for i in range(n)), "maximize")

    # Weight constraint
    model.addCons(quicksum(weights[i] * x[i] for i in range(n)) <= capacity)

    # Solve
    model.optimize()

    # Extract solution
    solution = {i: model.getVal(x[i]) for i in range(n)}
    total_value = sum(values[i] * solution[i] for i in range(n))
    total_weight = sum(weights[i] * solution[i] for i in range(n))
    
    return solution, total_value, total_weight


# Solve instance using LP relaxation

In [5]:
#Let's solve the LP relaxation

def solve_knapsack_lp_relaxation(weights, values, capacity):
    n = len(weights)
    model = Model("Knapsack_LP_Relax")

    # LP variables: 0 <= x[i] <= 1 (continuous)
    x = {i: model.addVar(lb=0.0, ub=1.0, vtype="C", name=f"x_{i}") for i in range(n)}

    # Maximize total value
    model.setObjective(quicksum(values[i] * x[i] for i in range(n)), "maximize")

    # Capacity constraint
    model.addCons(quicksum(weights[i] * x[i] for i in range(n)) <= capacity)

    # Solve
    model.optimize()

    # Extract (possibly fractional) solution
    sol = {i: model.getVal(x[i]) for i in range(n)}
    total_value = sum(values[i] * sol[i] for i in range(n))
    total_weight = sum(weights[i] * sol[i] for i in range(n))
    return sol, total_value, total_weight


In [6]:
print("Item Weights:", weights)
print("Item Values :", values)
print("Knapsack Capacity:", capacity)

solution, total_value, total_weight = solve_knapsack(weights, values, capacity)

print("\nChosen Fractions (LP relaxation):")
for i in range(len(weights)):
    print(f"Item {i}: x={solution[i]:.4f}, weight={weights[i]}, value={values[i]}")
print(f"\nTotal Weight = {total_weight:.4f}")
print(f"Total Value  = {total_value:.4f}")


# -----------------------------
# Run everything
# -----------------------------

solution, total_value, total_weight = solve_knapsack_lp_relaxation(weights, values, capacity)

print("\nChosen Fractions (LP relaxation):")
for i in range(len(weights)):
    print(f"Item {i}: x={solution[i]:.4f}, weight={weights[i]}, value={values[i]}")
print(f"\nTotal Weight = {total_weight:.4f}")
print(f"Total Value  = {total_value:.4f}")


Item Weights: [15, 19, 17, 2, 8]
Item Values : [19, 43, 41, 43, 14]
Knapsack Capacity: 36
feasible solution found by trivial heuristic after 0.0 seconds, objective value 0.000000e+00
presolving:

Chosen Fractions (LP relaxation):
Item 0: x=1.0000, weight=15, value=19
Item 1: x=1.0000, weight=19, value=43
Item 2: x=-0.0000, weight=17, value=41
Item 3: x=1.0000, weight=2, value=43
Item 4: x=-0.0000, weight=8, value=14

Total Weight = 36.0000
Total Value  = 105.0000
   (0.0s) running MILP presolver
   (0.0s) MILP presolver found nothing
(round 1, exhaustive) 0 del vars, 0 del conss, 0 add conss, 0 chg bounds, 0 chg sides, 0 chg coeffs, 1 upgd conss, 0 impls, 0 clqs
(round 2, fast)       0 del vars, 0 del conss, 0 add conss, 0 chg bounds, 1 chg sides, 1 chg coeffs, 1 upgd conss, 0 impls, 0 clqs
(round 3, medium)     0 del vars, 1 del conss, 0 add conss, 5 chg bounds, 1 chg sides, 1 chg coeffs, 1 upgd conss, 0 impls, 0 clqs
presolving (4 rounds: 4 fast, 3 medium, 2 exhaustive):
 5 deleted v

# Let's identify the optimal solution through enumeration

In [7]:
def solve_model_w_constraints(weights, values, capacity, constraints):
    
    n = len(weights)
    model = Model("Knapsack_LP_Relax")

    # LP variables: 0 <= x[i] <= 1 (continuous)
    x = {i: model.addVar(lb=0.0, ub=1.0, vtype="C", name=f"x_{i}") for i in range(n)}

    # Maximize total value
    model.setObjective(quicksum(values[i] * x[i] for i in range(n)), "maximize")

    # Capacity constraint
    model.addCons(quicksum(weights[i] * x[i] for i in range(n)) <= capacity)
    
    #Additional constraints:
    for var, constraint_val in constraints.items():
        model.addCons(x[var] == constraint_val)

    # Solve
    model.optimize()
    
    status = model.getStatus()
    
    if status == "optimal":
        # Extract (possibly fractional) solution
        sol = {i: model.getVal(x[i]) for i in range(n)}
        total_value = sum(values[i] * sol[i] for i in range(n))
        total_weight = sum(weights[i] * sol[i] for i in range(n))
        return sol, total_value, total_weight
    else:
        return None, None, None


In [8]:
def find_optimal_value(weights, values, capacity):
    V = []
    A = []
    A.append({}) #add root node
    while len(A) > 0:
        node = A.pop() #node is just the constraints of that node
        sol, total_value, total_weight = solve_model_w_constraints(weights, values, capacity, node) #solve LP relaxation of node
        if sol == None: #pop next node if this one is infeasible
            continue
        elif all(float(sol_val).is_integer() for sol_val in sol.values()): #if integer feasible
                V.append(total_value)
        else: #if fractional relaxation
            
            #identify which var to branch on
            fractional_keys = [key for key, v in sol.items() if abs(v - round(v)) > 1e-9]
            branch_var = fractional_keys[0]
            
            #add two new nodes to A
            node1 = node.copy()
            node2 = node.copy()
            node1[branch_var] = 1
            node2[branch_var] = 0
            A.append(node1)
            A.append(node2)
    
    print(f"{max(V)} IS THE OPTIMAL VALUE ")
    return max(V)
            
            
            
            
            
        
    

In [12]:
def find_optimal_value_with_dataset(weights, values, capacity):
    V = []
    A = []

    dataset = []         # list of rows (dicts) for each traversed node
    next_node_id = 0

    # root node
    root = {
        "constraints": {},    # your old node
        "depth": 0,
        "parent_id": None,
        "node_id": next_node_id,
    }
    next_node_id += 1
    A.append(root)

    while len(A) > 0:
        node = A.pop()

        constraints = node["constraints"]
        depth = node["depth"]
        parent_id = node["parent_id"]
        node_id = node["node_id"]

        # solve LP relaxation at this node
        sol, total_value, total_weight = solve_model_w_constraints(
            weights, values, capacity, constraints
        )

        # base row for this node
        row = {
            "node_id": node_id,
            "parent_id": parent_id,
            "depth": depth,
            "constraints": constraints.copy(),  # optional but often useful
        }

        # Case A: infeasible
        if sol is None:
            row["status"] = "infeasible"
            row["objective"] = None
            dataset.append(row)
            continue

        # we have a feasible LP relaxation
        # store the LP relaxation objective
        row["lp_objective"] = total_value

        # check integrality
        is_integer = all(float(sol_val).is_integer() for sol_val in sol.values())

        if is_integer:
            # Case B: integer feasible
            row["status"] = "integer"
            row["objective"] = total_value
            dataset.append(row)

            V.append(total_value)
        else:
            # Case D: fractional relaxation
            row["status"] = "fractional"
            row["objective"] = None
            dataset.append(row)

            # identify which var to branch on
            fractional_keys = [
                key for key, v in sol.items()
                if abs(v - round(v)) > 1e-9
            ]
            if not fractional_keys:
                # numerically weird but basically integer → treat as integer
                row["status"] = "integer"
                row["objective"] = total_value
                V.append(total_value)
                continue

            branch_var = fractional_keys[0]

            # add two new nodes to A (children)
            for fixed_value in [1, 0]:
                new_constraints = constraints.copy()
                new_constraints[branch_var] = fixed_value

                child = {
                    "constraints": new_constraints,
                    "depth": depth + 1,
                    "parent_id": node_id,
                    "node_id": next_node_id,
                }
                next_node_id += 1

                A.append(child)

    # sanity: handle case where no integer solution was found
    if not V:
        print("No integer-feasible solution found.")
        best_val = None
    else:
        best_val = max(V)
        print(f"{best_val} IS THE OPTIMAL VALUE")

    return best_val, dataset


In [13]:
import pandas as pd
import numpy as np

def build_node_dataframe(dataset, num_vars):
    rows = []
    for record in dataset:
        row = {}

        # Basic fields
        row["node_id"] = record["node_id"]
        row["parent_id"] = record["parent_id"]  # root will be None → becomes NaN in DataFrame
        row["depth"] = record["depth"]
        row["lp_objective"] = record.get("lp_objective", np.nan)
        row["status"] = record.get("status", None)

        # Initialize all variable columns to NaN
        for j in range(num_vars):
            row[f"x{j}"] = np.nan

        # Fill in constraints for this node
        # constraints is something like {1: 0, 2: 1}
        constraints = record.get("constraints", {})
        for var_idx, val in constraints.items():
            row[f"x{var_idx}"] = val

        rows.append(row)

    df = pd.DataFrame(rows)
    return df


In [15]:
def compute_optimal_subtree_flags(dataset, best_val):
    """
    dataset: list of node dicts (same as before)
    best_val: optimal integer objective value (e.g., from find_optimal_value_with_dataset)
    returns: dict node_id -> bool (whether its subtree contains an optimal solution)
    """

    # Build a parent map: node_id -> parent_id
    parent = {rec["node_id"]: rec["parent_id"] for rec in dataset}

    # Identify which nodes are optimal integer leaves
    is_opt_leaf = {
        rec["node_id"]: (
            rec.get("status") == "integer" and rec.get("objective") == best_val
        )
        for rec in dataset
    }

    # Initialize all flags to False
    has_optimal_in_subtree = {nid: False for nid in parent.keys()}

    # For each optimal leaf, mark it and all its ancestors
    for nid, is_leaf in is_opt_leaf.items():
        if not is_leaf:
            continue

        cur = nid
        # Walk up until root (parent_id = None) or already marked
        while cur is not None and not has_optimal_in_subtree[cur]:
            has_optimal_in_subtree[cur] = True
            cur = parent[cur]

    return has_optimal_in_subtree


In [16]:
best_val, dataset = find_optimal_value_with_dataset(weights, values, capacity)
n_vars = len(weights)

df = build_node_dataframe(dataset, n_vars)
opt_flags = compute_optimal_subtree_flags(dataset, best_val)

df["has_optimal_in_subtree"] = df["node_id"].map(opt_flags)

df

feasible solution found by trivial heuristic after 0.0 seconds, objective value 0.000000e+00
presolving:
(round 1, fast)       4 del vars, 0 del conss, 0 add conss, 0 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 0 clqs
(round 2, fast)       4 del vars, 1 del conss, 0 add conss, 1 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 0 clqs
presolving (3 rounds: 3 fast, 1 medium, 1 exhaustive):
 5 deleted vars, 1 deleted constraints, 0 added constraints, 1 tightened bounds, 0 added holes, 0 changed sides, 0 changed coefficients
 0 implications, 0 cliques
transformed 1/2 original solutions to the transformed problem space
Presolving Time: 0.00

SCIP Status        : problem is solved [optimal solution found]
Solving Time (sec) : 0.01
Solving Nodes      : 0
Primal Bound       : +1.22473684210526e+02 (2 solutions)
Dual Bound         : +1.22473684210526e+02
Gap                : 0.00 %
feasible solution found by trivial heuristic after 0.0 seconds, objective value 0.0

,node_id,parent_id,depth,lp_objective,status,x0,x1,x2,x3,x4,has_optimal_in_subtree
0,0,NaN,0,122.473684,fractional,NaN,NaN,NaN,NaN,NaN,True
1,2,0.0,1,109.400000,fractional,NaN,0.0,NaN,NaN,NaN,False
2,4,2.0,2,98.000000,integer,0.0,0.0,NaN,NaN,NaN,False
3,3,2.0,2,106.500000,fractional,1.0,0.0,NaN,NaN,NaN,False
4,6,3.0,3,103.000000,integer,1.0,0.0,NaN,NaN,0.0,False
5,5,3.0,3,102.529412,fractional,1.0,0.0,NaN,NaN,1.0,False
6,8,5.0,4,76.000000,integer,1.0,0.0,0.0,NaN,1.0,False
7,7,5.0,4,NaN,infeasible,1.0,0.0,1.0,NaN,1.0,False
8,1,0.0,1,122.176471,fractional,NaN,1.0,NaN,NaN,NaN,True
9,10,1.0,2,108.866667,fractional,NaN,1.0,0.0,NaN,NaN,True


, 0 chg coeffs, 0 upgd conss, 0 impls, 0 clqs
presolving (2 rounds: 2 fast, 1 medium, 1 exhaustive):
 3 deleted vars, 4 deleted constraints, 0 added constraints, 3 tightened bounds, 0 added holes, 0 changed sides, 0 changed coefficients
 0 implications, 0 cliques
presolved problem has 2 variables (0 bin, 0 int, 0 impl, 2 cont) and 0 constraints
Presolving Time: 0.00

 time | node  | left  |LP iter|LP it/n|mem/heur|mdpt |vars |cons |rows |cuts |sepa|confs|strbr|  dualbound   | primalbound  |  gap   | compl. 
t 0.0s|     1 |     0 |     0 |     - | trivial|   0 |   2 |   0 |   0 |   0 |  0 |   0 |   0 | 1.030000e+02 | 1.030000e+02 |   0.00%| unknown

SCIP Status        : problem is solved [optimal solution found]
Solving Time (sec) : 0.00
Solving Nodes      : 1
Primal Bound       : +1.03000000000000e+02 (1 solutions)
Dual Bound         : +1.03000000000000e+02
Gap                : 0.00 %
presolving:
(round 1, fast)       3 del vars, 3 del conss, 0 add conss, 3 chg bounds, 0 chg sides, 0 c